In [6]:
!pip install PyMuPDF

import fitz  # PyMuPDF
from google.colab import files

# Upload PDF
uploaded = files.upload()
pdf_path = next(iter(uploaded))

# Extract text
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join(page.get_text() for page in doc)

raw_text = extract_text_from_pdf(pdf_path)
print("Sample text:\n", raw_text[:1000])  # Preview first 1000 characters


Saving Financial report analysis.pdf to Financial report analysis.pdf
Sample text:
 Summary of Zenith Technologies Inc. Quarterly Report (Q1 2025) 
Key Financial Metrics 
●​ Net Profit: USD 842 million, up from USD 720 million in Q1 2024.​
 
●​ Earnings Per Share (EPS): USD 1.32, a 12% YoY increase from USD 1.18.​
 
●​ Return on Equity (ROE): 14.7%, exceeding the long-term average of 12%.​
 
●​ Cost/Income Ratio: Improved to 54.6%, showing enhanced operational efficiency.​
 
●​ Free Cash Flow: USD 1.2 billion, up 16% YoY.​
 
Income and Expenses 
●​ Revenue: USD 5.6 billion, a 9% YoY increase driven by growth in enterprise software 
sales.​
 
●​ Gross Margin: Held steady at 62.1% due to improved supply chain optimization.​
 
●​ Operating Expenses: Rose 5% YoY to USD 2.3 billion, primarily due to increased R&D 
investment and expansion into Asia-Pacific markets.​
 
●​ Research & Development Spend: USD 980 million (17.5% of revenue), reflecting 
strategic product innovation.​
 
Balance Sh

In [7]:
# Split into chunks
def chunk_text(text, max_words=200):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = chunk_text(raw_text)
print(f"Total chunks: {len(chunks)}")
print("\nSample chunk:\n", chunks[0])


Total chunks: 2

Sample chunk:
 Summary of Zenith Technologies Inc. Quarterly Report (Q1 2025) Key Financial Metrics ●​ Net Profit: USD 842 million, up from USD 720 million in Q1 2024.​ ●​ Earnings Per Share (EPS): USD 1.32, a 12% YoY increase from USD 1.18.​ ●​ Return on Equity (ROE): 14.7%, exceeding the long-term average of 12%.​ ●​ Cost/Income Ratio: Improved to 54.6%, showing enhanced operational efficiency.​ ●​ Free Cash Flow: USD 1.2 billion, up 16% YoY.​ Income and Expenses ●​ Revenue: USD 5.6 billion, a 9% YoY increase driven by growth in enterprise software sales.​ ●​ Gross Margin: Held steady at 62.1% due to improved supply chain optimization.​ ●​ Operating Expenses: Rose 5% YoY to USD 2.3 billion, primarily due to increased R&D investment and expansion into Asia-Pacific markets.​ ●​ Research & Development Spend: USD 980 million (17.5% of revenue), reflecting strategic product innovation.​ Balance Sheet Highlights ●​ Total Assets: USD 22.7 billion, up from USD 20.9 billion i

In [8]:
!pip install sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed chunks
embeddings = embedder.encode(chunks, show_progress_bar=True)
embeddings = np.array(embeddings)

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("FAISS index built with", len(embeddings), "vectors")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index built with 2 vectors


In [9]:
# Define question
question = "What is the main contribution of the paper?"

# Embed question
q_embedding = embedder.encode([question])

# Search top-k chunks
D, I = index.search(q_embedding, k=3)
retrieved_chunks = [chunks[i] for i in I[0]]

print("Top retrieved chunks:\n")
for i, chunk in enumerate(retrieved_chunks, 1):
    print(f"\nChunk {i}:\n{'-'*40}\n{chunk}")


Top retrieved chunks:


Chunk 1:
----------------------------------------
Summary of Zenith Technologies Inc. Quarterly Report (Q1 2025) Key Financial Metrics ●​ Net Profit: USD 842 million, up from USD 720 million in Q1 2024.​ ●​ Earnings Per Share (EPS): USD 1.32, a 12% YoY increase from USD 1.18.​ ●​ Return on Equity (ROE): 14.7%, exceeding the long-term average of 12%.​ ●​ Cost/Income Ratio: Improved to 54.6%, showing enhanced operational efficiency.​ ●​ Free Cash Flow: USD 1.2 billion, up 16% YoY.​ Income and Expenses ●​ Revenue: USD 5.6 billion, a 9% YoY increase driven by growth in enterprise software sales.​ ●​ Gross Margin: Held steady at 62.1% due to improved supply chain optimization.​ ●​ Operating Expenses: Rose 5% YoY to USD 2.3 billion, primarily due to increased R&D investment and expansion into Asia-Pacific markets.​ ●​ Research & Development Spend: USD 980 million (17.5% of revenue), reflecting strategic product innovation.​ Balance Sheet Highlights ●​ Total Assets: US

In [10]:
!pip install transformers

from transformers import pipeline

# Load FLAN-T5 model
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

# Prepare prompt
context = "\n".join(retrieved_chunks)
prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"

# Generate answer
result = qa_pipeline(prompt, max_length=200, do_sample=False)[0]['generated_text']
print("Answer:\n", result)


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (938 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer:
 This summary reflects Zenith Technologies Inc.’s strong Q1 2025 performance, driven by strategic innovation, prudent financial management, and a continued focus on emerging technologies and sustainability.


In [11]:
print("Source chunks used for the answer:\n")
for idx, chunk in enumerate(retrieved_chunks, 1):
    print(f"\n[Source {idx}]:\n{chunk}")


Source chunks used for the answer:


[Source 1]:
Summary of Zenith Technologies Inc. Quarterly Report (Q1 2025) Key Financial Metrics ●​ Net Profit: USD 842 million, up from USD 720 million in Q1 2024.​ ●​ Earnings Per Share (EPS): USD 1.32, a 12% YoY increase from USD 1.18.​ ●​ Return on Equity (ROE): 14.7%, exceeding the long-term average of 12%.​ ●​ Cost/Income Ratio: Improved to 54.6%, showing enhanced operational efficiency.​ ●​ Free Cash Flow: USD 1.2 billion, up 16% YoY.​ Income and Expenses ●​ Revenue: USD 5.6 billion, a 9% YoY increase driven by growth in enterprise software sales.​ ●​ Gross Margin: Held steady at 62.1% due to improved supply chain optimization.​ ●​ Operating Expenses: Rose 5% YoY to USD 2.3 billion, primarily due to increased R&D investment and expansion into Asia-Pacific markets.​ ●​ Research & Development Spend: USD 980 million (17.5% of revenue), reflecting strategic product innovation.​ Balance Sheet Highlights ●​ Total Assets: USD 22.7 billion, up from U